In [2]:
import psycopg2
import pandas as pd
from tabulate import tabulate  # For table formatting

In [3]:
import psycopg2

conn = psycopg2.connect(
    dbname="northwind_traders",
    user="postgres",
    password="Reterro%%1234",
    host="localhost",
    port="5432"
)
conn.set_session(autocommit=True)
cursor = conn.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()
print("Sucess!!!! You are connected to - ", record, "\n")



Sucess!!!! You are connected to -  ('PostgreSQL 16.2, compiled by Visual C++ build 1937, 64-bit',) 



In [4]:
# Function to Run Queries & Display Results
def query_run(query):
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        
        # Convert to DataFrame
        df = pd.DataFrame(result, columns=column_names)
        
        # # Print as Pretty Table
        print(tabulate(df, headers='keys', tablefmt='psql'))  
        
        #return df  # Return DataFrame for further use
        
    except psycopg2.Error as e:
        print(f"Error: {e}")

In [5]:
query ="""
        SELECT
        table_name as name,
        table_type as type
        FROM information_schema.tables
        WHERE table_schema = 'public' AND table_type IN ('BASE TABLE', 'VIEW');;"""

query_run(query)

+----+------------------------+------------+
|    | name                   | type       |
|----+------------------------+------------|
|  0 | us_states              | BASE TABLE |
|  1 | customers              | BASE TABLE |
|  2 | orders                 | BASE TABLE |
|  3 | employees              | BASE TABLE |
|  4 | shippers               | BASE TABLE |
|  5 | products               | BASE TABLE |
|  6 | order_details          | BASE TABLE |
|  7 | categories             | BASE TABLE |
|  8 | suppliers              | BASE TABLE |
|  9 | region                 | BASE TABLE |
| 10 | territories            | BASE TABLE |
| 11 | employee_territories   | BASE TABLE |
| 12 | customer_demographics  | BASE TABLE |
| 13 | customer_customer_demo | BASE TABLE |
+----+------------------------+------------+


In [6]:
for table in ['us_states', 'customers', 'orders', 'employees', 'shippers', 'products', 'order_details', 'categories', 'suppliers']:
    query = f"SELECT * FROM {table} LIMIT 5;"
    print(f"\n\n{table.upper()}\n")
    print(query_run(query))



US_STATES

+----+------------+--------------+--------------+----------------+
|    |   state_id | state_name   | state_abbr   | state_region   |
|----+------------+--------------+--------------+----------------|
|  0 |          1 | Alabama      | AL           | south          |
|  1 |          2 | Alaska       | AK           | north          |
|  2 |          3 | Arizona      | AZ           | west           |
|  3 |          4 | Arkansas     | AR           | south          |
|  4 |          5 | California   | CA           | west           |
+----+------------+--------------+--------------+----------------+
None


CUSTOMERS

+----+---------------+------------------------------------+--------------------+----------------------+-------------------------------+-------------+----------+---------------+-----------+----------------+----------------+
|    | customer_id   | company_name                       | contact_name       | contact_title        | address                       | city   

Ranking Employee Total Order Performance

In [7]:
query="""
        SELECT 
            employee_id,
            COUNT(*) AS orders_count,
            DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) AS rank
        FROM orders
        GROUP BY employee_id
        """
        
query_run(query)

+----+---------------+----------------+--------+
|    |   employee_id |   orders_count |   rank |
|----+---------------+----------------+--------|
|  0 |             4 |            156 |      1 |
|  1 |             3 |            127 |      2 |
|  2 |             1 |            123 |      3 |
|  3 |             8 |            104 |      4 |
|  4 |             2 |             96 |      5 |
|  5 |             7 |             72 |      6 |
|  6 |             6 |             67 |      7 |
|  7 |             9 |             43 |      8 |
|  8 |             5 |             42 |      9 |
+----+---------------+----------------+--------+


Ranking Employee Sales Performance

In [8]:
query=""" 
        WITH total_emp_sales AS (
            SELECT
                orders.employee_id,
                emp.first_name || ' ' || emp.last_name AS employee_name,
                SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales
            FROM
                orders
            JOIN
                order_details o_d ON orders.order_id = o_d.order_id
            JOIN
                employees emp ON orders.employee_id = emp.employee_id
            
            GROUP BY
                orders.employee_id, employee_name
            ORDER BY
                total_sales DESC
        )
        SELECT
            employee_id,
            employee_name,
            total_sales,
            DENSE_RANK() OVER (ORDER BY total_sales DESC) AS rank
        FROM
            total_emp_sales
        """

query_run(query)

+----+---------------+------------------+---------------+--------+
|    |   employee_id | employee_name    |   total_sales |   rank |
|----+---------------+------------------+---------------+--------|
|  0 |             4 | Margaret Peacock |      232891   |      1 |
|  1 |             3 | Janet Leverling  |      202813   |      2 |
|  2 |             1 | Nancy Davolio    |      192108   |      3 |
|  3 |             2 | Andrew Fuller    |      166538   |      4 |
|  4 |             8 | Laura Callahan   |      126862   |      5 |
|  5 |             7 | Robert King      |      124568   |      6 |
|  6 |             9 | Anne Dodsworth   |       77308.1 |      7 |
|  7 |             6 | Michael Suyama   |       73913.1 |      8 |
|  8 |             5 | Steven Buchanan  |       68792.3 |      9 |
+----+---------------+------------------+---------------+--------+


 Running Total of Monthly Sales

In [9]:
query = """
        SELECT
            TO_CHAR(DATE_TRUNC('MONTH', order_date), 'YYYY-MM') AS month,
            SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales_of_month
        FROM
            orders
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
        GROUP BY
            month
        ORDER BY
            month ASC
    """
    
query_run(query)

+----+---------+------------------------+
|    | month   |   total_sales_of_month |
|----+---------+------------------------|
|  0 | 1996-07 |                27861.9 |
|  1 | 1996-08 |                25485.3 |
|  2 | 1996-09 |                26381.4 |
|  3 | 1996-10 |                37515.7 |
|  4 | 1996-11 |                45600   |
|  5 | 1996-12 |                45239.6 |
|  6 | 1997-01 |                61258.1 |
|  7 | 1997-02 |                38483.6 |
|  8 | 1997-03 |                38547.2 |
|  9 | 1997-04 |                53033   |
| 10 | 1997-05 |                53781.3 |
| 11 | 1997-06 |                36362.8 |
| 12 | 1997-07 |                51020.9 |
| 13 | 1997-08 |                47287.7 |
| 14 | 1997-09 |                55629.2 |
| 15 | 1997-10 |                66749.2 |
| 16 | 1997-11 |                43533.8 |
| 17 | 1997-12 |                71398.4 |
| 18 | 1998-01 |                94222.1 |
| 19 | 1998-02 |                99415.3 |
| 20 | 1998-03 |               104

Month-Over-Month Sales Growth

In [10]:
query="""
    WITH monthly_sales AS (
        SELECT
            TO_CHAR(DATE_TRUNC('MONTH', order_date), 'YYYY-MM') AS month,
            SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales_of_month
        FROM
            orders
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
        GROUP BY
            month
        ORDER BY
            month ASC
    )
    SELECT
        month,
        total_sales_of_month,
        LAG(total_sales_of_month) OVER (ORDER BY month) AS last_month_sales,
        total_sales_of_month - LAG(total_sales_of_month) OVER (ORDER BY month) AS sales_growth
    FROM
        monthly_sales
            """
query_run(query)

+----+---------+------------------------+--------------------+----------------+
|    | month   |   total_sales_of_month |   last_month_sales |   sales_growth |
|----+---------+------------------------+--------------------+----------------|
|  0 | 1996-07 |                27861.9 |              nan   |       nan      |
|  1 | 1996-08 |                25485.3 |            27861.9 |     -2376.62   |
|  2 | 1996-09 |                26381.4 |            25485.3 |       896.125  |
|  3 | 1996-10 |                37515.7 |            26381.4 |     11134.3    |
|  4 | 1996-11 |                45600   |            37515.7 |      8084.32   |
|  5 | 1996-12 |                45239.6 |            45600   |      -360.415  |
|  6 | 1997-01 |                61258.1 |            45239.6 |     16018.4    |
|  7 | 1997-02 |                38483.6 |            61258.1 |    -22774.4    |
|  8 | 1997-03 |                38547.2 |            38483.6 |        63.5852 |
|  9 | 1997-04 |                53033   

Identifying High-Value Customers

In [11]:
query = """
    WITH avg_sales AS(
        SELECT
            AVG( o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS avg_sales
        FROM
            orders
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
    ),
    
    total_sales AS (
        SELECT
            orders.customer_id AS customer_id,
            customer.company_name AS company_name,
            SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales 
        FROM
            orders
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
        JOIN
            customers customer ON orders.customer_id = customer.customer_id
        GROUP BY
            orders.customer_id, company_name
    )
    SELECT
        customer_id,
        company_name,
        total_sales    
    FROM
        avg_sales,
        total_sales
    WHERE
        total_sales > avg_sales
    ORDER BY
        total_sales DESC

            
"""

query_run(query)

+----+---------------+------------------------------------+---------------+
|    | customer_id   | company_name                       |   total_sales |
|----+---------------+------------------------------------+---------------|
|  0 | QUICK         | QUICK-Stop                         |     110277    |
|  1 | ERNSH         | Ernst Handel                       |     104875    |
|  2 | SAVEA         | Save-a-lot Markets                 |     104362    |
|  3 | RATTC         | Rattlesnake Canyon Grocery         |      51097.8  |
|  4 | HUNGO         | Hungry Owl All-Night Grocers       |      49979.9  |
|  5 | HANAR         | Hanari Carnes                      |      32841.4  |
|  6 | KOENE         | Königlich Essen                    |      30908.4  |
|  7 | FOLKO         | Folk och fä HB                     |      29567.6  |
|  8 | MEREP         | Mère Paillarde                     |      28872.2  |
|  9 | WHITC         | White Clover Markets               |      27363.6  |
| 10 | FRANK

Percentage of Sales for Each Category

In [27]:
query=""" 
    WITH category_sales AS (
        SELECT
            cat.category_name,
            SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales
        FROM
            orders 
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
        JOIN
            products p ON o_d.product_id = p.product_id
        JOIN
            categories cat ON p.category_id = cat.category_id
        GROUP BY
            cat.category_name
        
    )
    SELECT
        category_name,
        total_sales,
        ROUND(CAST(total_sales / SUM(total_sales) OVER () * 100 AS NUMERIC), 2) AS sales_percentage
    FROM
        category_sales
    ORDER BY
        total_sales DESC;

"""

query_run(query)

+----+-----------------+---------------+--------------------+
|    | category_name   |   total_sales |   sales_percentage |
|----+-----------------+---------------+--------------------|
|  0 | Beverages       |      267868   |              21.16 |
|  1 | Dairy Products  |      234507   |              18.53 |
|  2 | Confections     |      167357   |              13.22 |
|  3 | Meat/Poultry    |      163022   |              12.88 |
|  4 | Seafood         |      131262   |              10.37 |
|  5 | Condiments      |      106047   |               8.38 |
|  6 | Produce         |       99984.6 |               7.9  |
|  7 | Grains/Cereals  |       95744.6 |               7.56 |
+----+-----------------+---------------+--------------------+


Top Products Per Category

In [39]:
query = """
    WITH category_sales AS (
        SELECT
            cat.category_name,
            p.product_name,
            SUM(o_d.quantity * o_d.unit_price * (1 - o_d.discount)) AS total_sales
        FROM
            orders 
        JOIN
            order_details o_d ON orders.order_id = o_d.order_id
        JOIN
            products p ON o_d.product_id = p.product_id
        JOIN
            categories cat ON p.category_id = cat.category_id
        GROUP BY
            cat.category_name,p.product_name
    ),
    category_rank AS (
    SELECT
        category_name,
        product_name,
        total_sales,
        ROUND(CAST(total_sales / SUM(total_sales) OVER () * 100 AS NUMERIC), 2) AS sales_percentage,
        RANK() OVER (PARTITION BY category_name ORDER BY total_sales DESC) AS rank
    FROM
        category_sales
    ORDER BY
        category_name ASC, total_sales DESC
    )
    SELECT
        category_name,
        product_name,
        total_sales,
        sales_percentage,
        rank
    FROM
        category_rank
    WHERE
        rank <= 3
    
"""

query_run(query)

+----+-----------------+----------------------------------+---------------+--------------------+--------+
|    | category_name   | product_name                     |   total_sales |   sales_percentage |   rank |
|----+-----------------+----------------------------------+---------------+--------------------+--------|
|  0 | Beverages       | Côte de Blaye                    |      141397   |              11.17 |      1 |
|  1 | Beverages       | Ipoh Coffee                      |       23526.7 |               1.86 |      2 |
|  2 | Beverages       | Chang                            |       16356   |               1.29 |      3 |
|  3 | Condiments      | Vegie-spread                     |       16701.1 |               1.32 |      1 |
|  4 | Condiments      | Sirop d'érable                   |       14352.6 |               1.13 |      2 |
|  5 | Condiments      | Louisiana Fiery Hot Pepper Sauce |       13869.9 |               1.1  |      3 |
|  6 | Confections     | Tarte au sucre       